In [4]:
# ===============================================
# 🌦️ Génération des données externes de Paris
# (Météo, jours fériés, vacances scolaires, événements)
# ===============================================

import pandas as pd
import requests
from datetime import datetime, timedelta
import os

# ------------------------------------------------------------
# 📁 0️⃣ Création du dossier de sortie
# ------------------------------------------------------------
output_dir = "external_data"
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------------
# 1️⃣ Grille temporelle horaire
# ------------------------------------------------------------
start = datetime(2023, 10, 1)
end = datetime(2024, 11, 11, 23)
hours = pd.date_range(start=start, end=end, freq="H")

features = pd.DataFrame({"time": hours})
features["date"] = features["time"].dt.date
features["hour"] = features["time"].dt.hour

# ------------------------------------------------------------
# 2️⃣ Données météo (Open-Meteo)
# ------------------------------------------------------------
print("Téléchargement des données météo...")

try:
    meteo_params = {
        "latitude": 48.8566,
        "longitude": 2.3522,
        "start_date": start.strftime("%Y-%m-%d"),
        "end_date": end.strftime("%Y-%m-%d"),
        "hourly": "temperature_2m,precipitation,cloud_cover,wind_speed_10m",
        "timezone": "Europe/Paris",
    }

    r = requests.get("https://archive-api.open-meteo.com/v1/archive", params=meteo_params)
    r.raise_for_status()
    meteo = pd.DataFrame(r.json()["hourly"])
    meteo["time"] = pd.to_datetime(meteo["time"])
    meteo.to_csv(os.path.join(output_dir, "meteo_raw.csv"), index=False)

    features = features.merge(meteo, on="time", how="left")
except Exception as e:
    print("⚠️ Erreur téléchargement météo :", e)

# ------------------------------------------------------------
# 3️⃣ Jours fériés
# ------------------------------------------------------------
print("Téléchargement des jours fériés...")

try:
    feries_url = "https://etalab.github.io/jours-feries-france-data/csv/jours_feries_metropole.csv"
    feries = pd.read_csv(feries_url)
    feries["date"] = pd.to_datetime(feries["date"]).dt.date
    feries["is_holiday"] = 1
    feries.to_csv(os.path.join(output_dir, "jours_feries.csv"), index=False)

    features = features.merge(feries[["date", "is_holiday"]], on="date", how="left").fillna({"is_holiday": 0})
except Exception as e:
    print("⚠️ Erreur téléchargement jours fériés :", e)
    features["is_holiday"] = 0

# ------------------------------------------------------------
# 4️⃣ Vacances scolaires (Zone C)
# ------------------------------------------------------------
print("Téléchargement des vacances scolaires...")

try:
    vac_url = "https://data.education.gouv.fr/explore/dataset/fr-en-calendrier-scolaire/download/?format=csv"
    vac = pd.read_csv(vac_url, sep=";", engine="python", on_bad_lines="skip")
    vac = vac[vac["zones"].str.contains("Zone C", na=False)]
    vac["date_debut"] = pd.to_datetime(vac["date_debut"], errors="coerce")
    vac["date_fin"] = pd.to_datetime(vac["date_fin"], errors="coerce")
    vac = vac.dropna(subset=["date_debut", "date_fin"])
    vac.to_csv(os.path.join(output_dir, "vacances_zoneC.csv"), index=False)

    def in_vacation(date):
        for _, row in vac.iterrows():
            if row["date_debut"].date() <= date <= row["date_fin"].date():
                return 1
        return 0

    features["is_vacation"] = features["date"].apply(in_vacation)
except Exception as e:
    print("⚠️ Erreur téléchargement vacances :", e)
    features["is_vacation"] = 0

# ------------------------------------------------------------
# 5️⃣ Événements publics (Paris)
# ------------------------------------------------------------
print("Téléchargement des événements...")

try:
    events_url = "https://opendata.paris.fr/explore/dataset/que-faire-a-paris-/download/?format=csv"
    events = pd.read_csv(events_url, sep=";", engine="python", on_bad_lines="skip", low_memory=False)
    events["date_start"] = pd.to_datetime(events["date_start"], errors="coerce")
    events["date_end"] = pd.to_datetime(events["date_end"], errors="coerce")
    events = events.dropna(subset=["date_start", "date_end"])
    events = events[["date_start", "date_end", "address_text"]]
    events.to_csv(os.path.join(output_dir, "evenements_raw.csv"), index=False)

    def has_event(date):
        for _, row in events.iterrows():
            if row["date_start"].date() <= date <= row["date_end"].date():
                return 1
        return 0

    features["has_event"] = features["date"].apply(has_event)
except Exception as e:
    print("⚠️ Erreur téléchargement événements :", e)
    features["has_event"] = 0

# ------------------------------------------------------------
# 6️⃣ Sauvegarde finale
# ------------------------------------------------------------
output_path = os.path.join(output_dir, "external_features_paris.csv")
features.to_csv(output_path, index=False)
print(f"\n✅ Données externes fusionnées sauvegardées dans : {output_path}\n")

# Aperçu
print(features.head())


Téléchargement des données météo...
Téléchargement des jours fériés...
Téléchargement des vacances scolaires...
⚠️ Erreur téléchargement vacances : 'date_debut'
Téléchargement des événements...
⚠️ Erreur téléchargement événements : The 'low_memory' option is not supported with the 'python' engine

✅ Données externes fusionnées sauvegardées dans : external_data\external_features_paris.csv

                 time        date  hour  temperature_2m  precipitation  \
0 2023-10-01 00:00:00  2023-10-01     0            14.2            0.0   
1 2023-10-01 01:00:00  2023-10-01     1            13.9            0.0   
2 2023-10-01 02:00:00  2023-10-01     2            13.5            0.0   
3 2023-10-01 03:00:00  2023-10-01     3            13.2            0.0   
4 2023-10-01 04:00:00  2023-10-01     4            12.8            0.0   

   cloud_cover  wind_speed_10m  is_holiday  is_vacation  has_event  
0            0             7.5         0.0            0          0  
1            0           